In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv
import pprint
import xmltodict, json
from datetime import datetime
import matplotlib.pyplot as plt

pp = pprint.PrettyPrinter(indent=4)
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['figure.figsize'] = [10, 6]
pd.set_option('display.max_rows', 500)


In [15]:
def normalise_df(df):
    df_num = df.select_dtypes(include='number')
    df_norm = (df_num-df_num.min())/(df_num.max()-df_num.min())
    df_norm.xg_for = 1 - df_norm.xg_for
    df[df_norm.columns] = df_norm.round(2)
    return df

In [16]:
def get_fixtures_for_url(url, team):
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("",res.text),'lxml')
    all_tables = soup.findAll("tbody")
    columns = ["comp", "venue", "possession", "xg_for", "xg_against", "goals_for", "goals_against", "opponent"]
    string_columns = ["comp", "venue", "opponent"]
    fixtures_table = all_tables[1]
    
    pre_df_fixtures = dict()
    features_wanted = columns
    rows_fixtures = fixtures_table.find_all('tr')
    for row in rows_fixtures:
        if(row.find('th',{"scope":"row"}) != None):
            # get date first
            cell = row.find("th",{"data-stat": "date"})
            a = cell.text.strip().encode()
            text=a.decode("utf-8")
            if "date" in pre_df_fixtures:
                pre_df_fixtures["date"].append(text)
            else:
                pre_df_fixtures["date"] = [text]

            for f in features_wanted:
                
                cell = row.find("td",{"data-stat": f})
                a = cell.text.strip().encode()
                text=a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if f not in string_columns:
                    if "(" in text:
                        text = text.split(" ")[0]
                    text = float(text.replace(',',''))

                if f in pre_df_fixtures:
                    pre_df_fixtures[f].append(text)
                else:
                    pre_df_fixtures[f] = [text]
                pre_df_fixtures["team"] = team
    df_fixtures = pd.DataFrame.from_dict(pre_df_fixtures)
    df_fixtures["date"] = pd.to_datetime(df_fixtures["date"])
    df_fixtures = df_fixtures.set_index("date")
    return df_fixtures

In [17]:
pl_team_map = {
    "Arsenal": {
        "fbref_id": "18bb7c10",
        "fpl_id": 1
    },"Aston-Villa": {
        "fbref_id": "8602292d",
        "fpl_id": 2
    }, "Brentford": {
        "fbref_id": "cd051869",
        "fpl_id": 3
    },"Brighton-and-Hove-Albion": {
        "fbref_id": "d07537b9",
        "fpl_id": 4
    }, "Burnley": {
        "fbref_id": "943e8050",
        "fpl_id": 5
    }, "Chelsea": {
        "fbref_id": "cff3d9bb",
        "fpl_id": 6
    }, "Crystal-Palace": {
        "fbref_id": "47c64c55",
        "fpl_id": 7
    }, "Everton": {
        "fbref_id": "d3fd31cc",
        "fpl_id": 8
    }, "Leeds-United": {
        "fbref_id": "5bfb9659",
        "fpl_id": 10
    }, "Leicester-City": {
        "fbref_id": "a2d435b3",
        "fpl_id": 9
    }, "Liverpool": {
        "fbref_id": "822bd0ba",
        "fpl_id": 11
    }, "Manchester-City": {
        "fbref_id": "b8fd03ef",
        "fpl_id": 12
    }, "Manchester-United": {
        "fbref_id": "19538871",
        "fpl_id": 13
    }, "Newcastle-United": {
        "fbref_id": "b2b47a98",
        "fpl_id": 14
    }, "Norwich-City": {
        "fbref_id": "1c781004",
        "fpl_id": 15
    }, "Southampton": {
        "fbref_id": "33c895d4",
        "fpl_id": 16
    }, "Tottenham-Hotspur": {
        "fbref_id": "361ca564",
        "fpl_id": 17
    }, "Watford": {
        "fbref_id": "2abfe087",
        "fpl_id": 18
    }, "West-Ham-United": {
        "fbref_id": "7c21e445",
        "fpl_id": 19
    }, "Wolverhampton-Wanderers": {
        "fbref_id": "8cec06e1",
        "fpl_id": 20
    }
}

In [18]:
data_pull = pd.DataFrame()
for team in pl_team_map:
    url = "https://fbref.com/en/squads/{}/{}-Stats".format(pl_team_map[team]["fbref_id"], team)
    print(url)
    try:
        team_fix = get_fixtures_for_url(url, " ".join(team.split('-')))
    except:
        print("Issue finding results for {}".format(team))
    data_pull = pd.concat([data_pull, team_fix])

https://fbref.com/en/squads/18bb7c10/Arsenal-Stats
https://fbref.com/en/squads/8602292d/Aston-Villa-Stats
https://fbref.com/en/squads/cd051869/Brentford-Stats
https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats
https://fbref.com/en/squads/943e8050/Burnley-Stats
https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats
https://fbref.com/en/squads/d3fd31cc/Everton-Stats
https://fbref.com/en/squads/5bfb9659/Leeds-United-Stats
https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats
https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
https://fbref.com/en/squads/19538871/Manchester-United-Stats
https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats
https://fbref.com/en/squads/1c781004/Norwich-City-Stats
https://fbref.com/en/squads/33c895d4/Southampton-Stats
https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats
https://fbref.com/en/squads/2abfe087/Watfor

In [19]:
aggregate_df = data_pull[data_pull.comp == "Premier League"]
aggregate_df = aggregate_df[aggregate_df.possession > 0]

In [20]:
att_ceil = 2.5
def_ceil = 2.5
aggregate_df.xg_for = aggregate_df.xg_for.apply(lambda diff: diff if np.abs(diff) < att_ceil else att_ceil)
aggregate_df.xg_against = aggregate_df.xg_against.apply(lambda diff: diff if np.abs(diff) < def_ceil else def_ceil)

aggregate_df["xg_diff"] = aggregate_df["xg_for"] - aggregate_df["xg_against"]


In [21]:
aggregate_df = normalise_df(aggregate_df)


In [22]:
aggregate_df

,comp,team,venue,possession,xg_for,xg_against,goals_for,goals_against,opponent,xg_diff
date,,,,,,,,,,
2021-08-13,Premier League,Arsenal,Away,0.77,0.46,0.50,0.00,0.29,Brentford,0.52
2021-08-22,Premier League,Arsenal,Home,0.25,0.92,1.00,0.00,0.29,Chelsea,0.04
2021-08-28,Premier League,Arsenal,Away,0.00,1.00,1.00,0.00,0.71,Manchester City,0.00
2021-09-11,Premier League,Arsenal,Home,0.53,0.00,0.21,0.14,0.00,Norwich City,0.90
2021-09-18,Premier League,Arsenal,Away,0.57,0.54,0.50,0.14,0.00,Burnley,0.48
2021-09-26,Premier League,Arsenal,Home,0.43,0.71,0.38,0.43,0.14,Tottenham,0.46
2021-10-02,Premier League,Arsenal,Away,0.35,0.83,0.54,0.00,0.00,Brighton,0.31
2021-10-18,Premier League,Arsenal,Home,0.58,0.46,0.33,0.29,0.29,Crystal Palace,0.60
2021-10-22,Premier League,Arsenal,Home,0.57,0.00,0.54,0.43,0.14,Aston Villa,0.73


In [23]:
home_agg = aggregate_df[aggregate_df.venue == "Home"].groupby("team").mean().round(2)
away_agg = aggregate_df[aggregate_df.venue == "Away"].groupby("team").mean().round(2)



In [24]:
team_strength = dict()
for team, ids in pl_team_map.items():
    team = team.replace("-", " ")
    fpl_id = ids["fpl_id"]
    team_strength[fpl_id] = dict()
    team_strength[fpl_id]["home"] = {
        "attack": away_agg.loc[team].xg_for,
        "defense": away_agg.loc[team].xg_against
    }
    team_strength[fpl_id]["away"] = {
        "attack": home_agg.loc[team].xg_for,
        "defense": home_agg.loc[team].xg_against
    }
    

In [25]:
with open("xg_diff_team_strength", 'w+') as f:
    f.write(json.dumps(team_strength))